In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

# Path to the original float32 model
input_model_path = "/content/cnn_model.onnx"

# Path to save the quantized int8 model
output_model_path = "classifier_int8.onnx"

# Perform dynamic quantization
quantize_dynamic(
    model_input=input_model_path,
    model_output=output_model_path,
    weight_type=QuantType.QInt8  # Or QuantType.QUInt8 for unsigned
)

print("✅ Quantization complete. Saved to:", output_model_path)


✅ Quantization complete. Saved to: classifier_int8.onnx


In [ ]:
pip install onnxruntime


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 358.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 709.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 615.6 kB/s eta 0:00:00


In [ ]:
pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 100.5 MB/s eta 0:00:00


In [ ]:
import onnx

model = onnx.load("/content/auto_int8.onnx")

# Check initializers (weights)
for tensor in model.graph.initializer:
    print(f"{tensor.name}: {onnx.TensorProto.DataType.Name(tensor.data_type)}")

# Optional: Check input/output types too
for input in model.graph.input:
    print(f"Input {input.name}: {input.type}")
for output in model.graph.output:
    print(f"Output {output.name}: {output.type}")


functional_1/conv2d_4_1/Squeeze:0: FLOAT
functional_1/conv2d_3_1/Squeeze:0: FLOAT
functional_1/conv2d_2_1/Squeeze:0: FLOAT
functional_1/conv2d_1_2/Squeeze:0: FLOAT
functional_1/conv2d_1/Squeeze:0: FLOAT
const_fold_opt__55__58: INT64
const_fold_opt__51: INT64
const_fold_opt__47: INT64
const_fold_opt__45: INT64
const_fold_opt__43__57: INT64
const_fold_opt__42: INT64
functional_1/conv2d_1/convolution/ReadVariableOp:0_scale: FLOAT
functional_1/conv2d_1/convolution/ReadVariableOp:0_zero_point: INT8
functional_1/conv2d_1/convolution/ReadVariableOp:0_quantized: INT8
functional_1/conv2d_1/BiasAdd:0_bias_reshape_shape: INT64
functional_1/conv2d_1_2/convolution/ReadVariableOp:0_scale: FLOAT
functional_1/conv2d_1_2/convolution/ReadVariableOp:0_zero_point: INT8
functional_1/conv2d_1_2/convolution/ReadVariableOp:0_quantized: INT8
functional_1/conv2d_1_2/BiasAdd:0_bias_reshape_shape: INT64
functional_1/conv2d_2_1/convolution/ReadVariableOp:0_scale: FLOAT
functional_1/conv2d_2_1/convolution/ReadVaria

In [ ]:
from onnxruntime.quantization import CalibrationDataReader
import numpy as np
import cv2
import os

class MyCalibrationDataReader(CalibrationDataReader):
    def __init__(self, image_dir, input_name):
        self.image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg'))]
        self.input_name = input_name
        self.data = []
        for img_path in self.image_paths:
            img = cv2.imread(img_path)
            img = cv2.resize(img, (64, 64))
            img = img.astype(np.float32) / 255.0  # adjust as per your normalization
            img = np.expand_dims(img, axis=0)
            self.data.append({self.input_name: img})
        self.idx = 0

    def get_next(self):
        if self.idx >= len(self.data):
            return None
        item = self.data[self.idx]
        self.idx += 1
        return item


In [ ]:
from onnxruntime.quantization import quantize_static, QuantType
import onnx

# Load ONNX model
model = onnx.load("/content/autoencoder_model.onnx")
input_name = model.graph.input[0].name

# Initialize reader
reader = MyCalibrationDataReader("path_to_sample_images", input_name)

quantize_static(
    model_input="/content/autoencoder_model.onnx",
    model_output="auto_static_int8.onnx",
    calibration_data_reader=reader,
    quant_format="QOperator",
    activation_type=QuantType.QInt8,
    weight_type=QuantType.QInt8,
    optimize_model=True
)


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_sample_images'

In [ ]:
# 2. Unzip the dataset
import zipfile
import os

zip_path = '/content/onnx_.zip'         # <- Your zip file path
extract_path = '/content/onnx_calibration_images'       # <- Extract location

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extracted files:", os.listdir(extract_path)[:5])  # Print some filenames to check


Extracted files: ['onnx_']


In [ ]:
# Define the CalibrationDataReader
from onnxruntime.quantization import CalibrationDataReader
import numpy as np
import cv2

class MyCalibrationDataReader(CalibrationDataReader):
    def __init__(self, image_dir, input_name):
        self.image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg'))]
        self.input_name = input_name
        self.data = []
        for img_path in self.image_paths:
            img = cv2.imread(img_path)
            img = cv2.resize(img, (64, 64))
            img = img.astype(np.float32) / 255.0
            img = np.expand_dims(img, axis=0)   # add batch dim (NCHW)
            self.data.append({self.input_name: img})
        self.idx = 0

    def get_next(self):
        if self.idx >= len(self.data):
            return None
        item = self.data[self.idx]
        self.idx += 1
        return item


In [ ]:
# Perform Static Quantization
from onnxruntime.quantization import quantize_static, QuantType
import onnx

# Load the original ONNX model
model = onnx.load("/content/cnn_model.onnx")
input_name = model.graph.input[0].name

# Initialize calibration data reader
reader = MyCalibrationDataReader("/content/onnx_calibration_images/onnx_", input_name)

# Run quantization
quantize_static(
    model_input="/content/cnn_model.onnx",
    model_output="/content/classifier_static_int8.onnx",
    calibration_data_reader=reader,
    quant_format="QOperator",
    activation_type=QuantType.QInt8,
    weight_type=QuantType.QInt8
)


print("Quantized model saved as classifier_static_int8.onnx")


Quantized model saved as classifier_static_int8.onnx


In [ ]:

import onnx

model = onnx.load("/content/classifier_static_int8.onnx")

# Check initializers (weights)
for tensor in model.graph.initializer:
    print(f"{tensor.name}: {onnx.TensorProto.DataType.Name(tensor.data_type)}")

# Optional: Check input/output types too
for input in model.graph.input:
    print(f"Input {input.name}: {input.type}")
for output in model.graph.output:
    print(f"Output {output.name}: {output.type}")

const_fold_opt__77: INT64
sequential_1/conv2d_5_1/BiasAdd__64:0_zero_point: INT8
sequential_1/conv2d_5_1/BiasAdd__64:0_scale: FLOAT
sequential_1/conv2d_5_1/convolution/ReadVariableOp:0_zero_point: INT8
sequential_1/conv2d_5_1/convolution/ReadVariableOp:0_scale: FLOAT
sequential_1/conv2d_5_1/convolution/ReadVariableOp:0_quantized: INT8
sequential_1/conv2d_5_1/Relu:0_zero_point: INT8
sequential_1/conv2d_5_1/Relu:0_scale: FLOAT
sequential_1/conv2d_6_1/convolution/ReadVariableOp:0_zero_point: INT8
sequential_1/conv2d_6_1/convolution/ReadVariableOp:0_scale: FLOAT
sequential_1/conv2d_6_1/convolution/ReadVariableOp:0_quantized: INT8
sequential_1/conv2d_6_1/Relu:0_zero_point: INT8
sequential_1/conv2d_6_1/Relu:0_scale: FLOAT
sequential_1/conv2d_7_1/convolution/ReadVariableOp:0_zero_point: INT8
sequential_1/conv2d_7_1/convolution/ReadVariableOp:0_scale: FLOAT
sequential_1/conv2d_7_1/convolution/ReadVariableOp:0_quantized: INT8
sequential_1/conv2d_7_1/Relu:0_zero_point: INT8
sequential_1/conv2d_7

In [4]:
import onnx

# Load the model
model_path = "/content/auto_int8.onnx"  # Change to your actual model path
model = onnx.load(model_path)

def inspect_operator_dtypes(model):
    fp32_nodes = []
    int32_nodes = []
    int8_nodes = []
    other_nodes = []

    for node in model.graph.node:
        # Convert protobuf containers to lists first
        input_names = list(node.input)
        output_names = list(node.output)
        all_names = input_names + output_names

        # Check for known type indicators in names (common with quantized models)
        if any("int8" in name.lower() for name in all_names):
            int8_nodes.append(node)
        elif any("int32" in name.lower() for name in all_names):
            int32_nodes.append(node)
        elif any("float" in name.lower() or "fp32" in name.lower() for name in all_names):
            fp32_nodes.append(node)
        else:
            other_nodes.append(node)

    print(f"🧠 FP32 nodes: {len(fp32_nodes)}")
    print(f"🧮 INT32 nodes: {len(int32_nodes)}")
    print(f"📦 INT8 nodes: {len(int8_nodes)}")
    print(f"❓ Other nodes: {len(other_nodes)}")

    print("\n🔍 Sample FP32 Ops:")
    for node in fp32_nodes[:10]:  # show just first 10
        print(f" - {node.op_type} (name: {node.name})")

inspect_operator_dtypes(model)

🧠 FP32 nodes: 0
🧮 INT32 nodes: 0
📦 INT8 nodes: 0
❓ Other nodes: 60

🔍 Sample FP32 Ops:


In [2]:
pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 100.3 MB/s eta 0:00:00


In [5]:
import onnx

# Load the model
model_path = "/content/classifier_int8.onnx"  # Change to your actual model path
model = onnx.load(model_path)

def inspect_operator_dtypes(model):
    fp32_nodes = []
    int32_nodes = []
    int8_nodes = []
    other_nodes = []

    for node in model.graph.node:
        # Convert protobuf containers to lists first
        input_names = list(node.input)
        output_names = list(node.output)
        all_names = input_names + output_names

        # Check for known type indicators in names (common with quantized models)
        if any("int8" in name.lower() for name in all_names):
            int8_nodes.append(node)
        elif any("int32" in name.lower() for name in all_names):
            int32_nodes.append(node)
        elif any("float" in name.lower() or "fp32" in name.lower() for name in all_names):
            fp32_nodes.append(node)
        else:
            other_nodes.append(node)

    print(f"🧠 FP32 nodes: {len(fp32_nodes)}")
    print(f"🧮 INT32 nodes: {len(int32_nodes)}")
    print(f"📦 INT8 nodes: {len(int8_nodes)}")
    print(f"❓ Other nodes: {len(other_nodes)}")

    print("\n🔍 Sample FP32 Ops:")
    for node in fp32_nodes[:10]:  # show just first 10
        print(f" - {node.op_type} (name: {node.name})")

inspect_operator_dtypes(model)

🧠 FP32 nodes: 0
🧮 INT32 nodes: 0
📦 INT8 nodes: 0
❓ Other nodes: 44

🔍 Sample FP32 Ops:


In [6]:
import onnx

# Load the model
model_path = "/content/auto_static_int8.onnx"  # Change to your actual model path
model = onnx.load(model_path)

def inspect_operator_dtypes(model):
    fp32_nodes = []
    int32_nodes = []
    int8_nodes = []
    other_nodes = []

    for node in model.graph.node:
        # Convert protobuf containers to lists first
        input_names = list(node.input)
        output_names = list(node.output)
        all_names = input_names + output_names

        # Check for known type indicators in names (common with quantized models)
        if any("int8" in name.lower() for name in all_names):
            int8_nodes.append(node)
        elif any("int32" in name.lower() for name in all_names):
            int32_nodes.append(node)
        elif any("float" in name.lower() or "fp32" in name.lower() for name in all_names):
            fp32_nodes.append(node)
        else:
            other_nodes.append(node)

    print(f"🧠 FP32 nodes: {len(fp32_nodes)}")
    print(f"🧮 INT32 nodes: {len(int32_nodes)}")
    print(f"📦 INT8 nodes: {len(int8_nodes)}")
    print(f"❓ Other nodes: {len(other_nodes)}")

    print("\n🔍 Sample FP32 Ops:")
    for node in fp32_nodes[:10]:  # show just first 10
        print(f" - {node.op_type} (name: {node.name})")

inspect_operator_dtypes(model)

🧠 FP32 nodes: 0
🧮 INT32 nodes: 0
📦 INT8 nodes: 0
❓ Other nodes: 64

🔍 Sample FP32 Ops:


In [7]:
import onnx
from onnx import mapping

model_path = "/content/auto_int8.onnx"
model = onnx.load(model_path)

def dtype_str(tensor_type):
    """Return the data type name from ONNX tensor type enum."""
    return mapping.TENSOR_TYPE_TO_NP_TYPE.get(tensor_type, "UNKNOWN")

def print_tensor_dtypes(model):
    all_tensors = (
        list(model.graph.initializer) +
        list(model.graph.input) +
        list(model.graph.output) +
        list(model.graph.value_info)
    )

    dtype_counts = {}
    for tensor in all_tensors:
        try:
            # Handle initializer differently
            if hasattr(tensor, 'data_type'):
                dtype = tensor.data_type
            else:
                dtype = tensor.type.tensor_type.elem_type
        except:
            continue

        dtype_name = dtype_str(dtype)
        dtype_counts[dtype_name] = dtype_counts.get(dtype_name, 0) + 1

    print("📊 Tensor Data Type Counts:")
    for dtype, count in dtype_counts.items():
        print(f" - {dtype}: {count}")

print_tensor_dtypes(model)

📊 Tensor Data Type Counts:
 - float32: 41
 - int64: 11
 - int8: 10


In [8]:
import onnx
from onnx import mapping

model_path = "/content/auto_static_int8.onnx"
model = onnx.load(model_path)

def dtype_str(tensor_type):
    """Return the data type name from ONNX tensor type enum."""
    return mapping.TENSOR_TYPE_TO_NP_TYPE.get(tensor_type, "UNKNOWN")

def print_tensor_dtypes(model):
    all_tensors = (
        list(model.graph.initializer) +
        list(model.graph.input) +
        list(model.graph.output) +
        list(model.graph.value_info)
    )

    dtype_counts = {}
    for tensor in all_tensors:
        try:
            # Handle initializer differently
            if hasattr(tensor, 'data_type'):
                dtype = tensor.data_type
            else:
                dtype = tensor.type.tensor_type.elem_type
        except:
            continue

        dtype_name = dtype_str(dtype)
        dtype_counts[dtype_name] = dtype_counts.get(dtype_name, 0) + 1

    print("📊 Tensor Data Type Counts:")
    for dtype, count in dtype_counts.items():
        print(f" - {dtype}: {count}")

print_tensor_dtypes(model)

📊 Tensor Data Type Counts:
 - int64: 6
 - int8: 19
 - float32: 50
 - int32: 10


In [9]:
import onnx
from onnx import mapping

model_path = "/content/classifier_int8.onnx"
model = onnx.load(model_path)

def dtype_str(tensor_type):
    """Return the data type name from ONNX tensor type enum."""
    return mapping.TENSOR_TYPE_TO_NP_TYPE.get(tensor_type, "UNKNOWN")

def print_tensor_dtypes(model):
    all_tensors = (
        list(model.graph.initializer) +
        list(model.graph.input) +
        list(model.graph.output) +
        list(model.graph.value_info)
    )

    dtype_counts = {}
    for tensor in all_tensors:
        try:
            # Handle initializer differently
            if hasattr(tensor, 'data_type'):
                dtype = tensor.data_type
            else:
                dtype = tensor.type.tensor_type.elem_type
        except:
            continue

        dtype_name = dtype_str(dtype)
        dtype_counts[dtype_name] = dtype_counts.get(dtype_name, 0) + 1

    print("📊 Tensor Data Type Counts:")
    for dtype, count in dtype_counts.items():
        print(f" - {dtype}: {count}")

print_tensor_dtypes(model)

📊 Tensor Data Type Counts:
 - float32: 29
 - int64: 4
 - int8: 10


In [10]:
import onnx
from onnx import mapping

model_path = "/content/classifier_static_int8.onnx"
model = onnx.load(model_path)

def dtype_str(tensor_type):
    """Return the data type name from ONNX tensor type enum."""
    return mapping.TENSOR_TYPE_TO_NP_TYPE.get(tensor_type, "UNKNOWN")

def print_tensor_dtypes(model):
    all_tensors = (
        list(model.graph.initializer) +
        list(model.graph.input) +
        list(model.graph.output) +
        list(model.graph.value_info)
    )

    dtype_counts = {}
    for tensor in all_tensors:
        try:
            # Handle initializer differently
            if hasattr(tensor, 'data_type'):
                dtype = tensor.data_type
            else:
                dtype = tensor.type.tensor_type.elem_type
        except:
            continue

        dtype_name = dtype_str(dtype)
        dtype_counts[dtype_name] = dtype_counts.get(dtype_name, 0) + 1

    print("📊 Tensor Data Type Counts:")
    for dtype, count in dtype_counts.items():
        print(f" - {dtype}: {count}")

print_tensor_dtypes(model)

📊 Tensor Data Type Counts:
 - int64: 1
 - int8: 17
 - float32: 34
 - int32: 10


In [15]:
import os
import zipfile
import numpy as np
import onnxruntime as ort
import psutil
import logging
from PIL import Image
import time
import matplotlib.pyplot as plt
from memory_profiler import memory_usage
from sklearn.metrics import accuracy_score

# Suppress logging
logging.getLogger('onnxruntime').setLevel(logging.ERROR)

# Define label dictionary (example)
label_dict = {0: "arm", 1: "elbow", 2: "face", 3: "foot", 4: "hand", 5: "leg", 6: "random"}

# Paths to your models
onnx_original = "/content/cnn_model.onnx"
onnx_quantized = "/content/classifier_int8.onnx"

# Helper to unzip folder
def unzip_folder(zip_path, extract_to="dataset"):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    return extract_to

# Preprocess image (resize and normalize)
def preprocess_image(image_path, size=(64, 64)):
    img = Image.open(image_path).convert("RGB").resize(size)
    img = np.array(img, dtype=np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Run inference on ONNX model
def run_onnx_inference(session, input_data):
    input_name = session.get_inputs()[0].name
    outputs = session.run(None, {input_name: input_data})
    return outputs[0]

# Measure inference time, RAM, and CPU usage
def measure_inference(func, *args):
    process = psutil.Process()
    start_time = time.time()
    start_cpu = process.cpu_percent(interval=None)
    mem_usage = memory_usage((func, args), interval=0.1, max_usage=True)
    result = func(*args)
    end_time = time.time()
    end_cpu = process.cpu_percent(interval=None)
    exec_time = (end_time - start_time) * 1000  # milliseconds
    normalized_cpu = ((end_cpu + start_cpu) / 2) / psutil.cpu_count()
    return mem_usage, normalized_cpu, exec_time, result

# Load ONNX models
def create_session(model_path):
    try:
        return ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])
    except Exception as e:
        print(f"Failed to load model: {e}")
        exit(1)

sess_orig = create_session(onnx_original)
sess_quant = create_session(onnx_quantized)

# Load image paths from unzipped folder
def load_images_from_folder(folder):
    images = []
    labels = []  # You should provide corresponding labels for the images
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder, filename)
            images.append(image_path)
            label = int(filename.split("_")[0])  # Assuming label is part of filename (e.g., "0_arm.jpg")
            labels.append(label)
    return images, labels

# Load images from folder
zip_path = "/content/onnx_.zip"  # Replace with your zip file path
dataset_folder = unzip_folder(zip_path)
image_paths, true_labels = load_images_from_folder(dataset_folder)

# Inference and evaluation
def evaluate_model(model_session, image_paths, true_labels):
    preds = []
    for img_path in image_paths:
        img_data = preprocess_image(img_path)
        pred = run_onnx_inference(model_session, img_data)
        preds.append(np.argmax(pred))  # Predicted class
    return accuracy_score(true_labels, preds)

# Evaluate original and quantized models
print("Evaluating Original Model...")
accuracy_orig = evaluate_model(sess_orig, image_paths, true_labels)
print(f"Original Model Accuracy: {accuracy_orig * 100:.2f}%")

print("Evaluating Quantized Model...")
accuracy_quant = evaluate_model(sess_quant, image_paths, true_labels)
print(f"Quantized Model Accuracy: {accuracy_quant * 100:.2f}%")

# Inference metrics for both models
def inference_metrics():
    print("\nOriginal Model Inference Metrics:")
    try:
        mem_orig, cpu_orig, time_orig, pred_orig = measure_inference(inference_original)
        print(f"  RAM={mem_orig:.2f} MB, CPU={cpu_orig:.2f}%, Time={time_orig:.2f} ms")
    except Exception as e:
        print(f"  Failed: {str(e)}")
        exit(1)

    print("\nQuantized Model Inference Metrics:")
    try:
        mem_quant, cpu_quant, time_quant, pred_quant = measure_inference(inference_quantized)
        print(f"  RAM={mem_quant:.2f} MB, CPU={cpu_quant:.2f}%, Time={time_quant:.2f} ms")
    except Exception as e:
        print(f"  Failed: {str(e)}")
        exit(1)

# Plot some examples (show original and predicted labels)
def plot_results(image_paths, true_labels, preds):
    plt.figure(figsize=(12, 6))
    for i in range(5):  # Show 5 examples
        plt.subplot(1, 5, i+1)
        img = Image.open(image_paths[i])
        plt.imshow(img)
        plt.title(f"True: {label_dict[true_labels[i]]}\nPred: {label_dict[preds[i]]}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Example usage of plotting the results for 5 images
plot_results(image_paths, true_labels, preds)

Failed to load model: [ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for ConvInteger(10) node with name 'sequential_1/conv2d_5_1/BiasAdd_quant'
Evaluating Original Model...
Original Model Accuracy: nan%
Evaluating Quantized Model...
Quantized Model Accuracy: nan%


/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


NameError: name 'preds' is not defined

In [14]:
pip install memory_profiler

In [16]:
import os
import zipfile
import numpy as np
import onnxruntime as ort
from sklearn.metrics import accuracy_score
from PIL import Image

# Define label dictionary (example)
label_dict = {0: "arm", 1: "elbow", 2: "face", 3: "foot", 4: "hand", 5: "leg", 6: "random"}

# Paths to your models
onnx_original = "/content/cnn_model.onnx"
onnx_quantized = "/content/classifier_static_int8.onnx"

# Helper to unzip folder
def unzip_folder(zip_path, extract_to="dataset"):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    return extract_to

# Preprocess image (resize and normalize)
def preprocess_image(image_path, size=(64, 64)):
    img = Image.open(image_path).convert("RGB").resize(size)
    img = np.array(img, dtype=np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Run inference on ONNX model
def run_onnx_inference(session, input_data):
    input_name = session.get_inputs()[0].name
    outputs = session.run(None, {input_name: input_data})
    return outputs[0]

# Load ONNX models
def create_session(model_path):
    try:
        return ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])
    except Exception as e:
        print(f"Failed to load model: {e}")
        exit(1)

sess_orig = create_session(onnx_original)
sess_quant = create_session(onnx_quantized)

# Load image paths from unzipped folder
def load_images_from_folder(folder):
    images = []
    labels = []  # You should provide corresponding labels for the images
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(folder, filename)
            images.append(image_path)
            label = int(filename.split("_")[0])  # Assuming label is part of filename (e.g., "0_arm.jpg")
            labels.append(label)
    return images, labels

# Load images from folder
zip_path = "/content/onnx_.zip"  # Replace with your zip file path
dataset_folder = unzip_folder(zip_path)
image_paths, true_labels = load_images_from_folder(dataset_folder)

# Evaluate model accuracy
def evaluate_model(model_session, image_paths, true_labels):
    preds = []
    for img_path in image_paths:
        img_data = preprocess_image(img_path)
        pred = run_onnx_inference(model_session, img_data)
        preds.append(np.argmax(pred))  # Predicted class
    accuracy = accuracy_score(true_labels, preds)
    return accuracy

# Evaluate original and quantized models
print("Evaluating Original Model...")
accuracy_orig = evaluate_model(sess_orig, image_paths, true_labels)
print(f"Original Model Accuracy: {accuracy_orig * 100:.2f}%")

print("Evaluating Quantized Model...")
accuracy_quant = evaluate_model(sess_quant, image_paths, true_labels)
print(f"Quantized Model Accuracy: {accuracy_quant * 100:.2f}%")

Evaluating Original Model...
Original Model Accuracy: nan%
Evaluating Quantized Model...
Quantized Model Accuracy: nan%


/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [19]:
import os
import zipfile
import numpy as np
import onnxruntime as ort
from sklearn.metrics import accuracy_score
from PIL import Image

# Define label dictionary (example)
label_dict = {0: "arm", 1: "elbow", 2: "face", 3: "foot", 4: "hand", 5: "leg", 6: "random"}

# Paths to your models
onnx_original = "/content/cnn_model.onnx"
onnx_quantized = "/content/classifier_static_int8.onnx"

# Helper to unzip folder
def unzip_folder(zip_path, extract_to="dataset"):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    return extract_to

# Preprocess image (resize and normalize)
def preprocess_image(image_path, size=(64, 64)):
    img = Image.open(image_path).convert("RGB").resize(size)
    img = np.array(img, dtype=np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Run inference on ONNX model
def run_onnx_inference(session, input_data):
    input_name = session.get_inputs()[0].name
    outputs = session.run(None, {input_name: input_data})
    return outputs[0]

# Load ONNX models
def create_session(model_path):
    try:
        return ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])
    except Exception as e:
        print(f"Failed to load model: {e}")
        exit(1)

sess_orig = create_session(onnx_original)
sess_quant = create_session(onnx_quantized)

# Load image paths and labels from unzipped folder
def load_images_from_folder(folder):
    images = []
    labels = []  # Labels are assigned based on subfolder names (e.g., '0_arm')
    class_labels = {cls: idx for idx, cls in label_dict.items()}

    for class_name in os.listdir(folder):
        class_folder = os.path.join(folder, class_name)

        if os.path.isdir(class_folder):
            for filename in os.listdir(class_folder):
                if filename.endswith(".jpg") or filename.endswith(".png"):
                    image_path = os.path.join(class_folder, filename)
                    images.append(image_path)
                    label = class_labels[class_name]  # Use folder name to assign label
                    labels.append(label)

    return images, labels

# Load images from folder
zip_path = "/content/natural_images.zip"  # Replace with your zip file path
dataset_folder = unzip_folder(zip_path)
image_paths, true_labels = load_images_from_folder(dataset_folder)

# Debug: Print out number of images and labels
print(f"Loaded {len(image_paths)} images.")
print(f"Loaded {len(true_labels)} labels.")

# Evaluate model accuracy
def evaluate_model(model_session, image_paths, true_labels):
    preds = []
    for img_path in image_paths:
        img_data = preprocess_image(img_path)
        pred = run_onnx_inference(model_session, img_data)

        # Debug: Check model output
        print(f"Prediction for {img_path}: {pred}")

        preds.append(np.argmax(pred))  # Predicted class

    # Debug: Check predictions and true labels
    print(f"Predictions: {preds}")
    print(f"True Labels: {true_labels}")

    accuracy = accuracy_score(true_labels, preds)
    return accuracy

# Evaluate original and quantized models
print("Evaluating Original Model...")
accuracy_orig = evaluate_model(sess_orig, image_paths, true_labels)
print(f"Original Model Accuracy: {accuracy_orig * 100:.2f}%")

print("Evaluating Quantized Model...")
accuracy_quant = evaluate_model(sess_quant, image_paths, true_labels)
print(f"Quantized Model Accuracy: {accuracy_quant * 100:.2f}%")

KeyError: 'onnx_'

In [20]:
import os
import zipfile
import numpy as np
import onnxruntime as ort
from sklearn.metrics import accuracy_score
from PIL import Image

# Define label dictionary (example)
label_dict = {0: "arm", 1: "elbow", 2: "face", 3: "foot", 4: "hand", 5: "leg", 6: "random"}

# Paths to your models
onnx_original = "/content/cnn_model.onnx"
onnx_quantized = "/content/classifier_int8.onnx"

# Helper to unzip folder
def unzip_folder(zip_path, extract_to="dataset"):
    if not os.path.exists(extract_to):
        os.makedirs(extract_to)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    return extract_to

# Preprocess image (resize and normalize)
def preprocess_image(image_path, size=(64, 64)):
    img = Image.open(image_path).convert("RGB").resize(size)
    img = np.array(img, dtype=np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Run inference on ONNX model
def run_onnx_inference(session, input_data):
    input_name = session.get_inputs()[0].name
    outputs = session.run(None, {input_name: input_data})
    return outputs[0]

# Load ONNX models
def create_session(model_path):
    try:
        return ort.InferenceSession(model_path, providers=['CPUExecutionProvider'])
    except Exception as e:
        print(f"Failed to load model: {e}")
        exit(1)

sess_orig = create_session(onnx_original)
sess_quant = create_session(onnx_quantized)

# Load image paths and labels from unzipped folder
def load_images_from_folder(folder):
    images = []
    labels = []  # Labels are assigned based on subfolder names (e.g., '0_arm')
    class_labels = {cls: idx for idx, cls in label_dict.items()}

    # Debug: Print out all folders inside the dataset
    print(f"Found folders: {os.listdir(folder)}")

    for class_name in os.listdir(folder):
        class_folder = os.path.join(folder, class_name)

        # Check if the folder is a directory and if it belongs to the label dictionary
        if os.path.isdir(class_folder):
            if class_name in class_labels:
                print(f"Processing class: {class_name}")
                for filename in os.listdir(class_folder):
                    if filename.endswith(".jpg") or filename.endswith(".png"):
                        image_path = os.path.join(class_folder, filename)
                        images.append(image_path)
                        label = class_labels[class_name]  # Use folder name to assign label
                        labels.append(label)
            else:
                print(f"Skipping unknown class folder: {class_name}")

    return images, labels

# Load images from folder
zip_path = "/content/natural_images.zip"  # Replace with your zip file path
dataset_folder = unzip_folder(zip_path)
image_paths, true_labels = load_images_from_folder(dataset_folder)

# Debug: Print out number of images and labels
print(f"Loaded {len(image_paths)} images.")
print(f"Loaded {len(true_labels)} labels.")

# Evaluate model accuracy
def evaluate_model(model_session, image_paths, true_labels):
    preds = []
    for img_path in image_paths:
        img_data = preprocess_image(img_path)
        pred = run_onnx_inference(model_session, img_data)

        # Debug: Check model output
        print(f"Prediction for {img_path}: {pred}")

        preds.append(np.argmax(pred))  # Predicted class

    # Debug: Check predictions and true labels
    print(f"Predictions: {preds}")
    print(f"True Labels: {true_labels}")

    accuracy = accuracy_score(true_labels, preds)
    return accuracy

# Evaluate original and quantized models
print("Evaluating Original Model...")
accuracy_orig = evaluate_model(sess_orig, image_paths, true_labels)
print(f"Original Model Accuracy: {accuracy_orig * 100:.2f}%")

print("Evaluating Quantized Model...")
accuracy_quant = evaluate_model(sess_quant, image_paths, true_labels)
print(f"Quantized Model Accuracy: {accuracy_quant * 100:.2f}%")

Failed to load model: [ONNXRuntimeError] : 9 : NOT_IMPLEMENTED : Could not find an implementation for ConvInteger(10) node with name 'sequential_1/conv2d_5_1/BiasAdd_quant'
Found folders: ['natural_images', 'onnx_']
Skipping unknown class folder: natural_images
Skipping unknown class folder: onnx_
Loaded 0 images.
Loaded 0 labels.
Evaluating Original Model...
Predictions: []
True Labels: []
Original Model Accuracy: nan%
Evaluating Quantized Model...
Predictions: []
True Labels: []
Quantized Model Accuracy: nan%


/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
